# Energy system model results

In [8]:
import logging
#logging.getLogger("imperative_model").setLevel(logging.DEBUG)
#logging.basicConfig(level=logging.DEBUG)

In [9]:
%load_ext autoreload
%autoreload 2
%run load_model.py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Check that processes and objects have been loaded correctly:

In [10]:
model.processes

[Process(id='CCGT', produces=['Electricity'], consumes=['NaturalGas'], has_stock=False),
 Process(id='ElectricCarUse', produces=['TransportService'], consumes=['Electricity'], has_stock=False),
 Process(id='HydrogenElectrolysis', produces=['Hydrogen'], consumes=['Electricity'], has_stock=False),
 Process(id='SteelProductionEAF', produces=['Steel'], consumes=['Electricity'], has_stock=False),
 Process(id='SteelProductionH2DRI', produces=['Steel'], consumes=['Hydrogen'], has_stock=False),
 Process(id='WindTurbine', produces=['Electricity'], consumes=[], has_stock=False)]

In [11]:
model.objects

[Object(id='Electricity', metric=rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Energy'), has_market=True),
 Object(id='Hydrogen', metric=rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Mass'), has_market=True),
 Object(id='NaturalGas', metric=rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Mass'), has_market=False),
 Object(id='Steel', metric=rdflib.term.URIRef('http://qudt.org/vocab/quantitykind/Mass'), has_market=False),
 Object(id='TransportService', metric=rdflib.term.URIRef('http://probs-lab.github.io/flowprog/metrics/PassengerKM'), has_market=False)]

We can now see the parametrised solution for all the flows in the system:

In [12]:
flows = solution_to_flows(model, {})
flows

,source,target,material,metric,value
0,CCGT,Electricity,Electricity,http://qudt.org/vocab/quantitykind/Energy,"Max(0, 5.6*Z_1*a_1 + 2.86*Z_1*(1 - a_1) + 2.3*..."
1,ElectricCarUse,TransportService,TransportService,http://probs-lab.github.io/flowprog/metrics/Pa...,Z_2
2,HydrogenElectrolysis,Hydrogen,Hydrogen,http://qudt.org/vocab/quantitykind/Mass,2.2*Z_1*(1 - a_1)
3,SteelProductionEAF,Steel,Steel,http://qudt.org/vocab/quantitykind/Mass,Z_1*a_1
4,SteelProductionH2DRI,Steel,Steel,http://qudt.org/vocab/quantitykind/Mass,Z_1*(1 - a_1)
5,WindTurbine,Electricity,Electricity,http://qudt.org/vocab/quantitykind/Energy,"1.0*Piecewise((0, 5.6*Z_1*a_1 + 2.86*Z_1*(1 - ..."
6,NaturalGas,CCGT,NaturalGas,http://qudt.org/vocab/quantitykind/Mass,"0.434782608695652*Max(0, 5.6*Z_1*a_1 + 2.86*Z_..."
7,Electricity,ElectricCarUse,Electricity,http://qudt.org/vocab/quantitykind/Energy,2.3*Z_2
8,Electricity,HydrogenElectrolysis,Electricity,http://qudt.org/vocab/quantitykind/Energy,2.86*Z_1*(1 - a_1)
9,Electricity,SteelProductionEAF,Electricity,http://qudt.org/vocab/quantitykind/Energy,5.6*Z_1*a_1


The `value` column uses the "natural" units for each material, which are a mix of mass, energy, and passenger-kilometers. So we can draw a Sankey diagram showing energy flows, we will calculate (or assign, for the purposes of illustration) the energy density of each material type:

In [16]:
energy_density = {
    "Electricity": 1,
    "TransportService": 0,
    "Hydrogen": recipe_data[model.U[0, 2]] / recipe_data[model.S[1, 2]],
    "NaturalGas": recipe_data[model.S[0, 0]] / recipe_data[model.U[2, 0]],
    "Steel": 0.1,
    
}

def solution_energy_density(values):
    flows = solution_to_flows(model, values).rename(columns={"value": "orig_value"})
    flows["value"] = flows["orig_value"] * flows["material"].map(lambda m: energy_density[m])
    return flows

flows = solution_energy_density({"Z_1": 10, "Z_2": 2, "S_1": 2, "a_1": 0.5})
flows

,source,target,material,metric,orig_value,value
0,CCGT,Electricity,Electricity,http://qudt.org/vocab/quantitykind/Energy,44.9000000000000,44.9000000000000
1,ElectricCarUse,TransportService,TransportService,http://probs-lab.github.io/flowprog/metrics/Pa...,2,0
2,HydrogenElectrolysis,Hydrogen,Hydrogen,http://qudt.org/vocab/quantitykind/Mass,11.0000000000000,14.3000000000000
3,SteelProductionEAF,Steel,Steel,http://qudt.org/vocab/quantitykind/Mass,5.00000000000000,0.500000000000000
4,SteelProductionH2DRI,Steel,Steel,http://qudt.org/vocab/quantitykind/Mass,5.00000000000000,0.500000000000000
5,WindTurbine,Electricity,Electricity,http://qudt.org/vocab/quantitykind/Energy,2.00000000000000,2.00000000000000
6,NaturalGas,CCGT,NaturalGas,http://qudt.org/vocab/quantitykind/Mass,19.5217391304348,44.9000000000000
7,Electricity,ElectricCarUse,Electricity,http://qudt.org/vocab/quantitykind/Energy,4.60000000000000,4.60000000000000
8,Electricity,HydrogenElectrolysis,Electricity,http://qudt.org/vocab/quantitykind/Energy,14.3000000000000,14.3000000000000
9,Electricity,SteelProductionEAF,Electricity,http://qudt.org/vocab/quantitykind/Energy,28.0000000000000,28.0000000000000


In [17]:
from ipysankeywidget import SankeyWidget
from ipywidgets import Layout
w = SankeyWidget(links=flows.to_dict(orient='records'), layout=Layout(width="1000", height="300"))
w.order = [
    ["NaturalGas"],
    ["CCGT", "WindTurbine"],
    ["Electricity"],
    ["HydrogenElectrolysis", "ElectricCarUse"],
    ["Hydrogen", "TransportService"],
    ["SteelProductionH2DRI", "SteelProductionEAF",],
    ["Steel"],
]
w

SankeyWidget(layout=Layout(height='300', width='1000'), links=[{'source': 'CCGT', 'target': 'Electricity', 'ma…

In the interactive notebook version, you can adjust the model parameters below and see how the Sankey diagram above is affected:

In [19]:
from ipywidgets import interact

@interact(d1=(0., 10.), d2=(0., 10.), s1=(0., 30.), a1=(0., 1.))
def calc_flows(d1=5.0, d2=2.0, s1=2.0, a1=0.5):
    flows = solution_energy_density({"Z_1": d1, "Z_2": d2, "S_1": s1, "a_1": a1})
    w.links = flows.to_dict(orient='records')

interactive(children=(FloatSlider(value=5.0, description='d1', max=10.0), FloatSlider(value=2.0, description='…

## Model definition history

To check how the model has been built, we can look at each variable and see what its equation looks like, with the "history" describing the steps in `load_model.py` which led to this:

In [20]:
from IPython.display import display

In [21]:
for j, sym in model.Y.items():
    print("Process output: %s" % model.processes[j].id)
    display(model[sym])
    print("History:", "\n".join(model.get_history(sym)))
    print("\n")

Process output: CCGT


Max(0, -S_0,5*Piecewise((0, U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4) <= 0), (S_1, S_1 <= U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4)), ((U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4))/S_0,5, True)) + U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4))/S_0,0

History: Supply from CCGT (second choice)


Process output: ElectricCarUse


Z_2/S_4,1

History: Demand for transport


Process output: HydrogenElectrolysis


...

History: Demand for steel


Process output: SteelProductionH2DRI


Z_1*(1 - a_1)/S_3,4

History: Demand for steel


Process output: WindTurbine


Piecewise((0, U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4) <= 0), (S_1, S_1 <= U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4)), ((U_0,1*Z_2/S_4,1 + U_0,3*Z_1*a_1/S_3,3 + U_0,2*U_1,4*Z_1*(1 - a_1)/(S_1,2*S_3,4))/S_0,5, True))

History: Supply from wind turbines (first choice)


